In [2]:
import asyncio
import os
from openai import AsyncOpenAI, OpenAIError

TIMEOUT_SECONDS = 30

model="gpt-4o-2024-05-13"

# Use the settings and environment variables

api_key = os.getenv('OPENAI_KEY')
if not api_key:
    raise OpenAIError("The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable.")
client = AsyncOpenAI(api_key=api_key)

# temperature=0

# Function to call GPT-4 API
async def get_gpt4_prediction(prompt, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Call the GPT-4 API
            response = await asyncio.wait_for(client.chat.completions.create(
                model=model,
                # temperature=temperature,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            ), timeout=TIMEOUT_SECONDS)
            # Extract the assistant's reply
            answer = response.choices[0].message.content.strip()
            return answer
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    print("Failed after retries.")
    return None

In [ ]:
from pathlib import Path
import pandas as pd

base_path = Path().resolve() / 'files'

meta = pd.read_csv(base_path / 'LiteratureCases_12_5_25.csv', index_col='id')

meta_df = pd.DataFrame(list(meta['omim']), columns=['OMIM_ID'])

# Convert meta['omim'] to a DataFrame if it's a Series
meta_df = pd.DataFrame(list(meta['omim']), columns=['OMIM_ID'])

# Add 'OMIM:' prefix to each OMIM ID in meta['omim']
meta_df['OMIM_ID'] = meta_df['OMIM_ID'].astype(str).apply(lambda x: 'OMIM:' + x)

# Filter df to include only rows that match meta['omim']
df_filtered = df[df['OMIM_ID'].isin(meta_df['OMIM_ID'])]

# Sample one row per OMIM_ID to ensure uniqueness
df_sampled = df_filtered.groupby('OMIM_ID').apply(lambda x: x.sample(1)).reset_index(drop=True)

# Merge with meta_df to ensure we keep only relevant OMIM_IDs
result = meta_df.merge(df_sampled, on='OMIM_ID', how='inner')
result

In [ ]:
# Dictionary to track used OMIM_IDs
used_indices = set()

# Function to sample one unique row per OMIM ID without repetition
def sample_unique_row(omim_id):
    global df_filtered, used_indices
    
    # Find available rows for this OMIM ID
    available_rows = df_filtered[(df_filtered['OMIM_ID'] == omim_id) & (~df_filtered.index.isin(used_indices))]
    
    if not available_rows.empty:
        # Select one row and mark it as used
        selected_row = available_rows.sample(1, random_state=42)
        used_indices.add(selected_row.index[0])
        return selected_row

    return None  # Return None if no available rows

# Apply the function to each OMIM ID in meta_df and collect results
sampled_rows = [sample_unique_row(omim_id) for omim_id in meta_df['OMIM_ID']]

# Concatenate all selected rows into a final DataFrame
result = pd.concat([row for row in sampled_rows if row is not None], ignore_index=True)
result

In [13]:
import random
from ast import literal_eval
from tqdm.auto import tqdm 

def get_random_hpo_synonym(hpo_file, hpo_id):
    synonyms = []
    term_name = None  # Store the base term name
    with open(hpo_file, 'r', encoding='utf-8') as file:
        inside_term = False
        current_id = None

        for line in file:
            line = line.strip()

            if line == "[Term]":  # Start of a new term
                inside_term = True
                current_id = None
                synonyms = []
                term_name = None  # Reset term name
                continue

            if inside_term:
                if line.startswith("id: "):  # Extract HPO ID
                    current_id = line.split("id: ")[1]

                elif line.startswith("name: "):  # Extract HPO term name
                    term_name = line.split("name: ")[1]

                elif line.startswith("synonym:"):  # Extract synonyms
                    synonym_text = line.split('"')[1]  # Extract synonym text
                    synonyms.append(synonym_text)

                elif line == "":  # End of current term
                    inside_term = False
                    if current_id == hpo_id:
                        choices = [term_name] + synonyms  # Include both base term and synonyms
                        return random.choice(choices) if choices else None  # Pick one randomly

    return None  # Return None if HPO ID is not found

# Example usage
hpo_file_path = base_path / "hp.obo"  # Path to your HPO file

hpo_synonyms = []
for hpos in tqdm(result['HPO_ID'].apply(literal_eval), total=result.shape[0]): 
    synonyms = []
    for hpo in hpos:
        random_synonym = get_random_hpo_synonym(hpo_file_path, hpo)
        if random_synonym:
            synonyms.append(random_synonym)
        else:
            print('No synonym')

            raise
    print(synonyms)
    hpo_synonyms.append(synonyms)


  0%|          | 0/416 [00:00<?, ?it/s]

['Generalised convulsion', 'Intellectual disability, moderate', 'Petit mal seizures', 'Ataxia', 'Infantile onset', 'Myoclonic seizure', 'Generalized clonic seizures']
['Bilateral tonic-clonic seizure', 'Moderate mental deficiency', 'Generalized non-motor (absence) seizure', 'Ataxia', 'Onset in infancy', 'Myoclonic seizure']
['Mild global developmental delay', 'Muscular hypotonia', 'Developmental delay, global', '2-hydroxyadipic aciduria', 'Alpha-aminoadipic aciduria', 'Attention deficit hyperactivity disorder']
['Seizure', 'Gait ataxia', 'Enlarged liver', 'Young adult onset', 'Low cholesterol esterification rate', 'Mental deficiency', 'Signs and symptoms begin before 15 years of age', 'Sea-blue histiocytosis']
['Seizures', 'Ataxic gait', 'Low cholesterol esterification rate', 'Increased spleen size', 'Intellectual disability', 'Juvenile onset', 'Sea-blue histiocyte']
['Epilepsy', 'Inability to coordinate movements when walking', 'Central hypotonia', 'Hepatomegaly', 'Low cholesterol est

In [14]:
result['HPO_Synonyms'] = hpo_synonyms
result

,Sr. No.,Disease,HPO_ID,HPO_Name,OMIM_ID,prompt,Case Description generated by DeepSeek,HPO_Synonyms
0,460,"epileptic encephalopathy, early infantile, 6 (...","['HP:0002069', 'HP:0002342', 'HP:0002121', 'HP...","['Bilateral tonic-clonic seizure', 'Intellectu...",OMIM:607208,Write a detailed clinical case description for...,NaN,"[Generalised convulsion, Intellectual disabili..."
1,461,"epileptic encephalopathy, early infantile, 6 (...","['HP:0002069', 'HP:0002342', 'HP:0002121', 'HP...","['Bilateral tonic-clonic seizure', 'Intellectu...",OMIM:607208,Write a detailed clinical case description for...,NaN,"[Bilateral tonic-clonic seizure, Moderate ment..."
2,11175,alpha-aminoadipic and alpha-ketoadipic aciduria,"['HP:0011342', 'HP:0001252', 'HP:0001263', 'HP...","['Mild global developmental delay', 'Hypotonia...",OMIM:204750,Write a detailed clinical case description for...,NaN,"[Mild global developmental delay, Muscular hyp..."
3,4593,"niemann-pick disease, type c1","['HP:0001250', 'HP:0002066', 'HP:0002240', 'HP...","['Seizure', 'Gait ataxia', 'Hepatomegaly', 'Yo...",OMIM:257220,Write a detailed clinical case description for...,NaN,"[Seizure, Gait ataxia, Enlarged liver, Young a..."
4,4594,"niemann-pick disease, type c1","['HP:0001250', 'HP:0002066', 'HP:0003349', 'HP...","['Seizure', 'Gait ataxia', 'Low cholesterol es...",OMIM:257220,Write a detailed clinical case description for...,NaN,"[Seizures, Ataxic gait, Low cholesterol esteri..."
...,...,...,...,...,...,...,...,...
411,13303,antley-bixler syndrome with genital anomalies ...,"['HP:0001176', 'HP:0003351', 'HP:0001769', 'HP...","['Large hands', 'Decreased circulating renin c...",OMIM:201750,Write a detailed clinical case description for...,NaN,"[Large hands, Low plasma renin activity, Broad..."
412,13302,antley-bixler syndrome with genital anomalies ...,"['HP:0000316', 'HP:0000013', 'HP:0000452', 'HP...","['Hypertelorism', 'Hypoplasia of the uterus', ...",OMIM:201750,Write a detailed clinical case description for...,NaN,"[Wide-set eyes, Small uterus, Coanal stenosis,..."
413,7840,"gaucher disease, perinatal lethal","['HP:0005257', 'HP:0011461', 'HP:0007479', 'HP...","['Thoracic hypoplasia', 'Fetal onset', 'Congen...",OMIM:608013,Write a detailed clinical case description for...,NaN,"[Small thorax, Foetal onset, Congenital lamell..."
414,7841,"gaucher disease, perinatal lethal","['HP:0011461', 'HP:0007479', 'HP:0002089', 'HP...","['Fetal onset', 'Congenital nonbullous ichthyo...",OMIM:608013,Write a detailed clinical case description for...,NaN,"[Foetal onset, Nonbullous congenital ichthyosi..."


In [15]:
print(result.iloc[0]['prompt'])

Write a detailed clinical case description for a patient following this template:
1. **Presentation:** Summarize the patient's presenting symptoms and clinical findings.
2. **Developmental History:** Describe any developmental delays (motor, speech, etc.).
3. **Neurological Examination:** Include relevant neurological findings (e.g., tone, IQ, etc.).
4. **Other Findings:** Mention any additional relevant clinical details (e.g., hearing status).
Ensure the description is comprehensive and structured.

The patient exhibits the following clinical features: Bilateral tonic-clonic seizure, Intellectual disability, moderate, Generalized non-motor (absence) seizure, Ataxia, Infantile onset, Myoclonic seizure, Generalized clonic seizure.


In [16]:
def replace_clinical_features(text, new_hpo_terms):
    # Find the position of "clinical features:"
    keyword = "clinical features:"
    index = text.find(keyword)

    if index != -1:
        # Keep only the text up to "clinical features:" and append the new terms
        updated_text = text[:index + len(keyword)] + " " + ", ".join(new_hpo_terms)
    else:
        # If "clinical features:" is not found, return the original text
        updated_text = text

    return updated_text

print(replace_clinical_features(result.iloc[0]['prompt'], result.iloc[0]['HPO_Synonyms']))

Write a detailed clinical case description for a patient following this template:
1. **Presentation:** Summarize the patient's presenting symptoms and clinical findings.
2. **Developmental History:** Describe any developmental delays (motor, speech, etc.).
3. **Neurological Examination:** Include relevant neurological findings (e.g., tone, IQ, etc.).
4. **Other Findings:** Mention any additional relevant clinical details (e.g., hearing status).
Ensure the description is comprehensive and structured.

The patient exhibits the following clinical features: Generalised convulsion, Intellectual disability, moderate, Petit mal seizures, Ataxia, Infantile onset, Myoclonic seizure, Generalized clonic seizures


In [34]:
print(result.iloc[0]['prompt'])

Write a detailed clinical case description for a patient following this template:
1. **Presentation:** Summarize the patient's presenting symptoms and clinical findings.
2. **Developmental History:** Describe any developmental delays (motor, speech, etc.).
3. **Neurological Examination:** Include relevant neurological findings (e.g., tone, IQ, etc.).
4. **Other Findings:** Mention any additional relevant clinical details (e.g., hearing status).
Ensure the description is comprehensive and structured.

The patient exhibits the following clinical features: Mild global developmental delay, Hypotonia, Global developmental delay, 2-hydroxyadipic aciduria, Alpha-aminoadipic aciduria, Attention deficit hyperactivity disorder.


In [12]:
df_filtered

,Sr. No.,Disease,HPO_ID,HPO_Name,OMIM_ID,prompt,Case Description generated by DeepSeek
189,190,hypermethioninemia due to adenosine kinase def...,"['HP:0001397', 'HP:0003235', 'HP:0002465', 'HP...","['Hepatic steatosis', 'Hypermethioninemia', 'P...",OMIM:614300,Write a detailed clinical case description for...,NaN
190,191,hypermethioninemia due to adenosine kinase def...,"['HP:0001397', 'HP:0003235', 'HP:0002465', 'HP...","['Hepatic steatosis', 'Hypermethioninemia', 'P...",OMIM:614300,Write a detailed clinical case description for...,NaN
191,192,hypermethioninemia due to adenosine kinase def...,"['HP:0003235', 'HP:0002465', 'HP:0001250', 'HP...","['Hypermethioninemia', 'Poor speech', 'Seizure...",OMIM:614300,Write a detailed clinical case description for...,NaN
192,193,hypermethioninemia due to adenosine kinase def...,"['HP:0001397', 'HP:0003235', 'HP:0002465', 'HP...","['Hepatic steatosis', 'Hypermethioninemia', 'P...",OMIM:614300,Write a detailed clinical case description for...,NaN
193,194,hypermethioninemia due to adenosine kinase def...,"['HP:0003235', 'HP:0002465', 'HP:0001396', 'HP...","['Hypermethioninemia', 'Poor speech', 'Cholest...",OMIM:614300,Write a detailed clinical case description for...,NaN
...,...,...,...,...,...,...,...
23707,23708,gyrate atrophy of choroid and retina with or w...,"['HP:0000662', 'HP:0030498', 'HP:0012026', 'HP...","['Nyctalopia', 'Macular thickening', 'Hyperorn...",OMIM:258870,Write a detailed clinical case description for...,NaN
23708,23709,gyrate atrophy of choroid and retina with or w...,"['HP:0000662', 'HP:0030498', 'HP:0012026', 'HP...","['Nyctalopia', 'Macular thickening', 'Hyperorn...",OMIM:258870,Write a detailed clinical case description for...,NaN
23709,23710,gyrate atrophy of choroid and retina with or w...,"['HP:0000662', 'HP:0030498', 'HP:0012026', 'HP...","['Nyctalopia', 'Macular thickening', 'Hyperorn...",OMIM:258870,Write a detailed clinical case description for...,NaN
23710,23711,gyrate atrophy of choroid and retina with or w...,"['HP:0000662', 'HP:0030498', 'HP:0012026', 'HP...","['Nyctalopia', 'Macular thickening', 'Hyperorn...",OMIM:258870,Write a detailed clinical case description for...,NaN


In [14]:
prompt = """
Write a detailed clinical case description for a patient following this format:

1. **Presentation:** Summarize the patient's presenting symptoms and clinical findings.
2. **Developmental History:** Describe any developmental delays (motor, speech, etc.).
3. **Neurological Examination:** Include relevant neurological findings (e.g., tone, IQ, etc.).
4. **Other Findings:** Mention any additional relevant clinical details (e.g., hearing status).

The patient exhibits the following clinical features: Epileptic spasm, Intellectual disability, profound, Primary microcephaly, EEG with burst suppression, Epileptic encephalopathy, Neonatal respiratory distress.

Please ensure you only use the clinical features provided and do not add any additional information.

Please write the clinical case description in a natural manner. Do not use formatting or bullet points.

Finally, do not suggest what the disease might be or how to intervene. Just describe the clinical features.
"""

answer = await asyncio.wait_for(get_gpt4_prediction(prompt), timeout=TIMEOUT_SECONDS)
print(answer)

A six-month-old male infant presented to the neurology clinic with a chief complaint of epileptic spasms that had been occurring multiple times daily. The parents reported that these spasms began when the child was around two months old. The spasms typically occurred in clusters, with the child appearing to suddenly flex forward at the waist with a simultaneous adduction of the arms. Additionally, the mother expressed concerns about her child's severe developmental delays. At this age, he had not yet achieved typical developmental milestones such as head control, rolling over, or even making cooing sounds.

From a developmental standpoint, the patient exhibited significant global delays. His motor development was notably behind schedule; he was unable to hold his head up independently or reach for objects. Speech development was also profoundly delayed; he had not begun to babble and did not respond to auditory stimuli in a manner typical for his age.

The neurological examination reve

In [17]:
from tqdm.auto import tqdm

outfit = """

Please ensure you only use the clinical features provided and do not add any additional information.

Please write the clinical case description in a natural manner. Do not use formatting or bullet points.

Finally, do not suggest what the disease might be or how to intervene. Just describe the clinical features.
"""

cases = result[['prompt', 'HPO_Synonyms']]

answers = []
for i, row in tqdm(enumerate(cases.iterrows()), total=len(cases)):

    prompt, new_hpo_terms = row[1]['prompt'], row[1]['HPO_Synonyms']
    prompt = prompt.replace("Ensure the description is comprehensive and structured.\n", "")
    prompt = replace_clinical_features(prompt, new_hpo_terms)
    prompt += outfit

    answer = await asyncio.wait_for(get_gpt4_prediction(prompt), timeout=TIMEOUT_SECONDS)

    print(answer)
    print('\n')
    answers.append(answer)

  0%|          | 0/416 [00:00<?, ?it/s]

A six-year-old boy presented to the pediatric neurology clinic with a history of generalized convulsions and intellectual disability. His parents reported that he began having seizures in infancy, initially characterized by myoclonic jerks and generalized clonic seizures. Over time, these evolved to include petit mal seizures, marked by brief lapses in consciousness without loss of posture. His seizure frequency has varied, with some periods of improvement and others of increased frequency, despite medical interventions.

Developmentally, his parents observed delays early on. He struggled with gross and fine motor skills, taking longer than his peers to sit, crawl, and walk. His speech development was also delayed; he began speaking in short phrases significantly later than expected for his age. His intellectual disability is classified as moderate, affecting his daily functioning and learning abilities.

During the neurological examination, evident ataxia was noted. His gait was unste

In [29]:
from utils.api_usage import get_relevant
from tqdm.auto import tqdm

cases['description'] = answers

description_hpos = {}
# for id, desc in tqdm(zip(df.index, df['caseDescription']), total=len(df)):
for id, desc in tqdm(zip(cases.index, cases['description']), total=len(cases)):
# for id, desc in tqdm(zip(df.index, df['caseDescription - Final']), total=len(df)):
    try:
        if id in description_hpos.keys():
            continue
        relevant_hpo_terms, related_hpo_terms = get_relevant(desc)
        description_hpos[id] = list(relevant_hpo_terms.keys())
    except Exception as e:
        print(f"Error processing case {id}: {e}")
        continue

description_hpos

  0%|          | 0/416 [00:00<?, ?it/s]

{0: ['HP:0000750',
  'HP:0001249',
  'HP:0001251',
  'HP:0001263',
  'HP:0002069',
  'HP:0002121',
  'HP:0002123',
  'HP:0002317',
  'HP:0002370'],
 1: ['HP:0000750',
  'HP:0001251',
  'HP:0001263',
  'HP:0002069',
  'HP:0002121',
  'HP:0002123',
  'HP:0002172',
  'HP:0002342'],
 2: ['HP:0000750',
  'HP:0001252',
  'HP:0001263',
  'HP:0001270',
  'HP:0001290',
  'HP:0007018',
  'HP:0011342',
  'HP:0410309'],
 3: ['HP:0001268',
  'HP:0001270',
  'HP:0001290',
  'HP:0001982',
  'HP:0002066',
  'HP:0002240',
  'HP:0002344',
  'HP:0003349',
  'HP:0006887',
  'HP:0007240'],
 4: ['HP:0001249',
  'HP:0001744',
  'HP:0001982',
  'HP:0002066',
  'HP:0002311',
  'HP:0003349'],
 5: ['HP:0000750',
  'HP:0001249',
  'HP:0001250',
  'HP:0001263',
  'HP:0001270',
  'HP:0001982',
  'HP:0002066',
  'HP:0002240',
  'HP:0003349',
  'HP:0011398'],
 6: ['HP:0000007',
  'HP:0001249',
  'HP:0001270',
  'HP:0001290',
  'HP:0001310',
  'HP:0001982',
  'HP:0002240',
  'HP:0002355',
  'HP:0003349'],
 7: ['HP:000

In [19]:
from utils.hpo_ontology import load_ontology

# Load the HPO ontology
graph, ic_dict = load_ontology(annotations='OMIM')

from oaklib import get_adapter

adapter = get_adapter(f"simpleobo:{base_path}/mondo.obo")

In [20]:
import json

with open(base_path / "case_description_hpos_gpt.json", "wb") as f:
    f.write(json.dumps(description_hpos).encode("utf-8"))

In [21]:
# Step 1: Load your JSON file
with open(base_path / "case_description_hpos_gpt.json", "r") as f:
    case_hpos = json.load(f)

# Step 2: Parse the hp.obo file to extract HPO code -> description mapping
hpo_mapping = {}
current_id = None
current_name = None

with open(base_path / "hp.obo", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line.startswith("[Term]"):  # New HPO entry starts
            current_id = None
            current_name = None
        elif line.startswith("id: HP:"):
            current_id = line.split(": ")[1]  # Extract HPO ID
        elif line.startswith("name:") and current_id:
            current_name = line.split(": ")[1]  # Extract HPO name
            hpo_mapping[current_id] = current_name  # Store in dictionary


# Step 3: Replace HPO codes with descriptions per case
converted_cases = {}
for case_id, hpo_codes in case_hpos.items():
    case_id_str = str(case_id)  # Ensure case ID is a string
    converted_cases[case_id_str] = {hpo_code: hpo_mapping.get(hpo_code, "Unknown term") for hpo_code in hpo_codes}

# Step 4: Save the output to a new JSON file
with open(base_path / "converted_cases_gpt.json", "w", encoding="utf-8") as f:
    json.dump(converted_cases, f, indent=4, ensure_ascii=False)

print("Conversion complete! Saved as converted_hpos.json.")

Conversion complete! Saved as converted_hpos.json.


In [22]:
import pickle

with open(base_path / "depth_cache.pkl", "rb") as f:
    depth_cache = pickle.load(f)


In [23]:
# Rename column
result.rename(columns={'OMIM_ID': 'OMIM', 'Disease': 'Correct Diagnosis'}, inplace=True)

# Remove 'OMIM:' prefix
result['OMIM'] = result['OMIM'].str.replace('OMIM:', '', regex=True)

In [24]:
result['description'] = answers

In [25]:
result.to_csv('gpt_case_descriptions.csv')

In [55]:
import utils.score_diagnosis 
import importlib
import utils.overlap_method

# make changes to example.py file
importlib.reload(utils.score_diagnosis)
importlib.reload(utils.overlap_method)

from utils.score_diagnosis import evaluate_predictions_and_save_logs, timed_evaluate

results_path = Path().resolve() / 'results'

# Initialize dictionary to store times
execution_times = {}

# TOM
_, execution_times["TOM_SA"] = timed_evaluate(
    evaluate_predictions_and_save_logs,
    relevant_hpos=description_hpos, 
    prompts_df=result, 
    adapter=adapter, 
    graph=graph, 
    ic_dict=ic_dict, 
    depth_cache=depth_cache,
    negative_hpos=None,
    weighted_score_active=False,
    semantic_similarity=True,
    log_file_path=results_path / "evaluation_log_gpt_descriptions.txt", 
    output_ranks_path=results_path / "evaluation_ranks_gpt_descriptions.npz"
)

  0%|          | 0/127 [00:00<?, ?it/s]

Logs saved to /Users/timhulshof/Documents/analysis_chatimd/results/evaluation_log_gpt_descriptions.txt
Ranks saved to /Users/timhulshof/Documents/analysis_chatimd/results/evaluation_ranks_gpt_descriptions.npz
Time taken: 765.24 seconds


In [26]:
import chromadb

chroma_client = chromadb.PersistentClient(
                                    path=str('/Users/timhulshof/Documents/test_chatimd_latest/chatimd_interface/backend/chatimd_backend/databases/hpo_synonym_db')
                                )

collection = chroma_client.get_collection(name="hpo_synonym")

new_client = chromadb.PersistentClient(
                                    path=str(base_path / 'synthetic_patients')
                                )

profiles = new_client.get_collection(name="synthetic_disease_profiles_v1")

In [30]:
import utils.score_diagnosis 
import importlib

# make changes to example.py file
importlib.reload(utils.score_diagnosis)

from utils.score_diagnosis import evaluate_predictions_and_save_logs, timed_evaluate

results_path = Path().resolve() / 'results'

# TOM
_, execution_times["TOM_SA_HNSW_WHOLE"] = timed_evaluate(
    evaluate_predictions_and_save_logs,
    relevant_hpos=description_hpos, 
    prompts_df=result, 
    adapter=adapter, 
    graph=graph, 
    ic_dict=ic_dict, 
    depth_cache=depth_cache,
    index=profiles,
    collection=collection,
    semantic_similarity=True,
    hnsw=True,
    k=6000,
    id_to_disease=None,
    weighted_score_active=True,
    log_file_path=results_path / "evaluation_log_TOM_SA_HNSW_gpt.txt", 
    output_ranks_path=results_path / "evaluation_ranks_TOM_SA_HNSW_gpt.npz"
)

  0%|          | 0/416 [00:00<?, ?it/s]

INFO:backoff:Backing off send_request(...) for 0.6s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
INFO:backoff:Backing off send_request(...) for 0.0s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
INFO:backoff:Backing off send_request(...) for 0.3s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
INFO:backoff:Backing off send_request(...) for 0.2s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
INFO:backoff:Backing off send_request(...) for 0.7s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Logs saved to /Users/timhulshof/Documents/analysis_chatimd/results/evaluation_log_TOM_SA_HNSW_gpt.txt
Ranks saved to /Users/timhulshof/Documents/analysis_chatimd/results/evaluation_ranks_TOM_SA_HNSW_gpt.npz
Time taken: 3095.53 seconds


NameError: name 'execution_times' is not defined